# Forward Operator

## Import

In [ ]:
import os.path as op
import numpy as np

import os
import mne
from mne.datasets import fetch_fsaverage
import mne_bids
import vtk

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
fsaverage = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

## Input Raw BIDS

In [ ]:
bids_path = mne_bids.BIDSPath(subject=subject,
                              session=session,
                              task=task,
                              datatype='eeg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path) 
raw.load_data() 
events, event_id = mne.events_from_annotations(raw) 
raw = raw.copy().pick_types(meg=False, eeg=True, eog=True)

### Interpolate Bads

In [ ]:
raw.info['bads'] = bad_list
raw = raw.copy().interpolate_bads(reset_bads=True)

## Set Montage

In [ ]:
print('######################## Setting montage. ########################')

In [ ]:
# Read and set the EEG electrode locations, which are already in fsaverage's
# space (MNI space) for standard_1020:
raw.rename_channels(mapping={'VREF' : 'Cz'}) 
montage = mne.channels.make_standard_montage('GSN-HydroCel-129')
#montage = mne.channels.make_dig_montage()
raw.set_montage(montage)
# It is strongly recommended to use average-reference-as-projection approach
#raw.set_eeg_reference(ref_channels=['E129'])  # needed for inverse modeling
raw.set_eeg_reference('average',projection=True)  # needed for inverse modeling

#### Plot the Montage

## Setup Source Space and Compute Forward Solution

In [ ]:
print('######################## Computing forward solution. ########################')

In [ ]:
fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
                                bem=bem, eeg=True, mindist=5.0, n_jobs=1)
print(fwd)

##### From here, standard forward and inverse operators can be computed

#### Plot the Forward Solution

# Write Forward Solution

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
forward_solution = "05_fwd"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> rawreports -> epoch_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, forward_solution)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, forward_solution))

In [ ]:
forward_path = os.path.join(root_path, "EGI_OUTPUTS\\"+subname+"\\"+sesname+"\\"+eeg_path+"\\"+forward_solution)
forward_title =  'sub-'+subject+'_ses-'+session+'_task-'+task+'_fwd.fif'
subject_forward_path = os.path.join(forward_path, forward_title)

In [ ]:
mne.write_forward_solution(subject_forward_path, fwd, overwrite=True)